In [2]:
import pandas as pd

df = pd.read_parquet("../data/yellow_tripdata_2025-01.parquet")

print(df.shape)
print(df.info())

(3475226, 20)
<class 'pandas.DataFrame'>
RangeIndex: 3475226 entries, 0 to 3475225
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     str           
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee        

In [4]:
threshold = 0.9 * len(df)
cols_to_drop = [col for col in df.columns if df[col].isnull().sum() > threshold]

df.drop(columns=cols_to_drop, inplace=True)
print('dropped columns:', cols_to_drop)

dropped columns: []


In [7]:
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_cols:
    df.fillna({col: 0}, inplace=True)

categorical_cols = df.select_dtypes(include=['str']).columns
for col in categorical_cols:
    df.fillna({col: 'Unknown'}, inplace=True)

In [8]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

df['VendorID'] = df['VendorID'].astype('category')
df['RatecodeID'] = df['RatecodeID'].astype('category')
df['payment_type'] = df['payment_type'].astype('category')

In [12]:
df['pickup_hour'] = df['tpep_pickup_datetime'].dt.hour
df['pickup_date'] = df['tpep_pickup_datetime'].dt.date
df['weekday'] = df['tpep_pickup_datetime'].dt.weekday
df['is_weekend'] = df['weekday'] >= 5

df['trip_duration_min'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds()/60
df['trip_speed_mph'] = df['trip_distance'] / (df['trip_duration_min']/60)
df.replace({'trip_speed_mph': {float('inf'): 0, -float('inf'): 0}}, inplace=True)
df.fillna({'trip_speed_mph': 0}, inplace=True)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee,pickup_hour,pickup_date,weekday,is_weekend,trip_duration_min,trip_speed_mph
0,1,2025-01-01 00:18:38,2025-01-01 00:26:59,1.0,1.60,1.0,N,229,237,1,...,18.00,2.5,0.0,0.00,0,2025-01-01,2,False,8.350000,11.497006
1,1,2025-01-01 00:32:40,2025-01-01 00:35:13,1.0,0.50,1.0,N,236,237,1,...,12.12,2.5,0.0,0.00,0,2025-01-01,2,False,2.550000,11.764706
2,1,2025-01-01 00:44:04,2025-01-01 00:46:01,1.0,0.60,1.0,N,141,141,1,...,12.10,2.5,0.0,0.00,0,2025-01-01,2,False,1.950000,18.461538
3,2,2025-01-01 00:14:27,2025-01-01 00:20:01,3.0,0.52,1.0,N,244,244,2,...,9.70,0.0,0.0,0.00,0,2025-01-01,2,False,5.566667,5.604790
4,2,2025-01-01 00:21:34,2025-01-01 00:25:06,3.0,0.66,1.0,N,244,116,2,...,8.30,0.0,0.0,0.00,0,2025-01-01,2,False,3.533333,11.207547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3475221,2,2025-01-31 23:01:48,2025-01-31 23:16:29,0.0,3.35,0.0,Unknown,79,237,0,...,20.60,0.0,0.0,0.75,23,2025-01-31,4,False,14.683333,13.688990
3475222,2,2025-01-31 23:50:29,2025-02-01 00:17:27,0.0,8.73,0.0,Unknown,161,116,0,...,32.89,0.0,0.0,0.75,23,2025-01-31,4,False,26.966667,19.423980
3475223,2,2025-01-31 23:26:59,2025-01-31 23:43:01,0.0,2.64,0.0,Unknown,144,246,0,...,19.66,0.0,0.0,0.75,23,2025-01-31,4,False,16.033333,9.879418
3475224,2,2025-01-31 23:14:34,2025-01-31 23:34:52,0.0,3.16,0.0,Unknown,142,107,0,...,22.30,0.0,0.0,0.75,23,2025-01-31,4,False,20.300000,9.339901


In [13]:
df = df[(df['trip_distance'] > 0) & (df['trip_duration_min'] > 0)]
df = df[df['trip_speed_mph'] > 100]

In [15]:
df.to_parquet('../data/clean_yellow_tripdata_2025-01.parquet', index=False)
df.to_csv('../data/clean_yellow_tripdata_2025-01.csv', index=False)

In [18]:
df.groupby('pickup_hour')['total_amount'].sum().reset_index().to_csv('../data/revenue_by_hour.csv', index=False)
df.groupby('PULocationID')['trip_distance'].mean().reset_index().to_csv('../data/avg_distance_by_zone.csv', index=False)
df.groupby('VendorID')['total_amount'].sum().reset_index().to_csv('../data/vendor_performance.csv', index=False)
df.groupby('is_weekend')['total_amount'].mean().reset_index().to_csv('../data/weekday_weekend_avg.csv', index=False)